In [1]:
import pandas as pd


# In[83]:
df = pd.read_csv('Data_Train.csv')
df.head(10)
print(df.shape)


(78458, 11)


In [2]:

#Replace K, M, B
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    if ',' in x:
        return float(x.replace(',', '')) 
    return float(x)

df.Likes = df.Likes.apply(value_to_float)
df.Popularity = df.Popularity.apply(value_to_float)
df.to_csv('Data_Train_Transformed_1.csv')
df = df[df['Views'] > df["Likes"]]
# print(df.shape)
# print(df.head())

In [3]:
df.isnull().sum()

Unique_ID     0
Name          0
Genre         0
Country       0
Song_Name     1
Timestamp     0
Views         0
Comments      0
Likes         0
Popularity    0
Followers     0
dtype: int64

In [4]:
# Distribution of features by Genre
df_mean = df.groupby('Genre').mean()
#df_mean

In [5]:
#Correlation matrix
df.drop(['Unique_ID','Country','Timestamp','Song_Name','Comments','Followers'],inplace = True, axis = 1)
df.head()


,Name,Genre,Views,Likes,Popularity
0,Hardstyle,danceedm,14017,499.0,97.0
1,Dj Aladdin,danceedm,1918,49.0,17.0
2,Maxximize On Air,danceedm,9668,312.0,91.0
3,GR6 EXPLODE,rbsoul,113036,2400.0,76.0
4,Tritonal,danceedm,110024,3031.0,699.0


In [6]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# Converting Genre and Name to labelencoding
labelencoder = LabelEncoder()
genreoneHotEncoder = OneHotEncoder(categories='auto')
nameoneHotEncoder = OneHotEncoder(categories='auto')
genre_transformed_data = genreoneHotEncoder.fit_transform((df['Genre'].values).reshape(-1,1))
genre_hashed_features = pd.DataFrame( genre_transformed_data.toarray()).add_prefix('gen_')
genre_hashed_features.head()

# oneHotEncoder = OneHotEncoder(categories='auto')
# name_transformed_data = nameoneHotEncoder.fit_transform((df['Name'].values).reshape(-1,1))
# name_hashed_features = pd.DataFrame( name_transformed_data.toarray()).add_prefix('name_')
# name_hashed_features.head()

,gen_0,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6,gen_7,gen_8,gen_9,...,gen_11,gen_12,gen_13,gen_14,gen_15,gen_16,gen_17,gen_18,gen_19,gen_20
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#Hash the Name
# from sklearn.feature_extraction import FeatureHasher
# fh = FeatureHasher(n_features=10, input_type='string')
# name_hashed_features = fh.fit_transform(df['Name'])
# name_hashed_features = name_hashed_features.toarray()
# name_hashed_features = pd.DataFrame(name_hashed_features)
# name_hashed_features.head()
# print(name_hashed_features.shape)

# from sklearn.feature_extraction import FeatureHasher
# fh = FeatureHasher(n_features=10, input_type='string')
# genre_hashed_features = fh.fit_transform(df['Genre'])
# genre_hashed_features = genre_hashed_features.toarray()
# genre_hashed_features = pd.DataFrame(genre_hashed_features,columns=['gen1','gen2','gen3','gen4','gen5','gen6','gen7','gen8','gen9','gen10',])
# genre_hashed_features.head()

print(genre_hashed_features.shape)

(77170, 21)


In [8]:
#Standard scaling
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
sc_x = StandardScaler()
ct = ColumnTransformer([
         ('somename', StandardScaler(), ['Likes', 'Popularity'])
       ], remainder='passthrough')
df = ct.fit_transform(df)
df = pd.DataFrame(df,columns=['Likes','Popularity','Name','Genre','Views'])
print(df.shape)

(77170, 5)


In [9]:
y = df['Views']
df.drop(['Views','Name','Genre'], inplace=True,axis=1)
y.plot.hist()

In [10]:
# df = df.join(name_hashed_features)
df = df.join(genre_hashed_features)
print(df.shape)
df['Likes'] = df['Likes'].apply(pd.to_numeric, errors='coerce')
df['Popularity'] = df['Popularity'].apply(pd.to_numeric, errors='coerce')

(77170, 23)


In [11]:
from sklearn.model_selection import train_test_split
#print(df_scaled)
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=.2, random_state=205)

In [12]:
#XGB Boost regression
import xgboost as xgb
import numpy as np
from sklearn import metrics
xlf = xgb.XGBRegressor()
xlf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])
# calculate the auc score
y_pred = xlf.predict(X_test)
#RMSE - Test
print("XGBoost - Linear regression: ", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))


[17:51:35] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:3.35707e+06


D:\Applns\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[1]	validation_0-rmse:3.03555e+06
[2]	validation_0-rmse:2.75289e+06
[3]	validation_0-rmse:2.50845e+06
[4]	validation_0-rmse:2.27444e+06
[5]	validation_0-rmse:2.0732e+06
[6]	validation_0-rmse:1.89272e+06
[7]	validation_0-rmse:1.72244e+06
[8]	validation_0-rmse:1.58427e+06
[9]	validation_0-rmse:1.45174e+06
[10]	validation_0-rmse:1.33829e+06
[11]	validation_0-rmse:1.23556e+06
[12]	validation_0-rmse:1.14884e+06
[13]	validation_0-rmse:1.06882e+06
[14]	validation_0-rmse:999867
[15]	validation_0-rmse:936997
[16]	validation_0-rmse:885166
[17]	validation_0-rmse:836342
[18]	validation_0-rmse:795925
[19]	validation_0-rmse:760088
[20]	validation_0-rmse:729412
[21]	validation_0-rmse:704566
[22]	validation_0-rmse:680796
[23]	validation_0-rmse:661684
[24]	validation_0-rmse:644213
[25]	validation_0-rmse:630884
[26]	validation_0-rmse:620630
[27]	validation_0-rmse:609445
[28]	validation_0-rmse:600040
[29]	validation_0-rmse:593720
[30]	validation_0-rmse:586181
[31]	validation_0-rmse:580627
[32]	validation

In [13]:
import pandas as pd
# In[83]:
test_df = pd.read_csv('D:\Hackathon\MachineHack - Music\Data_Test.csv')
test_df.head(10)
print(test_df.shape)

(19615, 10)


In [14]:
#Replace K, M, B
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    if ',' in x:
        return float(x.replace(',', '')) 
    return float(x)

test_df.Likes = test_df.Likes.apply(value_to_float)
test_df.Popularity = test_df.Popularity.apply(value_to_float)

In [15]:
#Correlation matrix
test_df.drop(['Unique_ID','Country','Timestamp','Song_Name','Comments','Followers'],inplace = True, axis = 1)
test_df.head()

,Name,Genre,Likes,Popularity
0,L2Share♫79,all-music,5428.0,118.0
1,Morgan Page,danceedm,738.0,195.0
2,Dirty Workz,danceedm,386.0,96.0
3,Gucci Mane,all-music,388.0,19.0
4,Angga Kecap,rock,79.0,1.0


In [16]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# Converting Genre and Name to labelencoding
labelencoder = LabelEncoder()
# test_df['Genre'] = labelencoder.fit_transform(test_df['Genre'])
#test_df['Name'] = labelencoder.fit_transform(test_df['Name'])
test_df.head()

#Hash the Name
# name_hashed_features = fh.transform(test_df['Name'])
# name_hashed_features = name_hashed_features.toarray()
# name_hashed_features = pd.DataFrame(name_hashed_features)
# name_hashed_features.head()
# print(name_hashed_features.shape)

genre_transformed_data = genreoneHotEncoder.transform((test_df['Genre'].values).reshape(-1,1))
genre_hashed_features = pd.DataFrame( genre_transformed_data.toarray()).add_prefix('gen_')
genre_hashed_features.head()

# name_transformed_data = nameoneHotEncoder.transform((test_df['Name'].values).reshape(-1,1))
# name_hashed_features = pd.DataFrame( name_transformed_data.toarray()).add_prefix('name_')
# name_hashed_features.head()

,gen_0,gen_1,gen_2,gen_3,gen_4,gen_5,gen_6,gen_7,gen_8,gen_9,...,gen_11,gen_12,gen_13,gen_14,gen_15,gen_16,gen_17,gen_18,gen_19,gen_20
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
test_df.drop(['Name','Genre'],inplace = True, axis = 1)
#Standard scaling
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
sc_x = StandardScaler()
ct = ColumnTransformer([
         ('somename', StandardScaler(), ['Likes', 'Popularity'])
       ], remainder='passthrough')
test_df = ct.fit_transform(test_df)
test_df = pd.DataFrame(test_df,columns=['Likes','Popularity'])
print(test_df.head())

      Likes  Popularity
0 -0.068006   -0.166740
1 -0.158250   -0.151475
2 -0.165023   -0.171102
3 -0.164984   -0.186368
4 -0.170930   -0.189936


In [18]:
# test_df = test_df.join(name_hashed_features)
test_df = test_df.join(genre_hashed_features)
# test_df['Genre'] = test_df['Genre'].apply(pd.to_numeric, errors='coerce')
test_df['Likes'] = test_df['Likes'].apply(pd.to_numeric, errors='coerce')
test_df['Popularity'] =test_df['Popularity'].apply(pd.to_numeric, errors='coerce')
print(test_df.shape)

(19615, 23)


In [21]:
# calculate the auc score
y_pred_1 = xlf.predict(test_df)
y_pred_1 = pd.DataFrame(y_pred_1)
y_pred_1.to_csv('Data_Test_PredictedOne_Hot_Genre_No_Name_XGBoost_V_L_P.csv',sep='#')